UK trade in goods by industry, country and commodity, exports

In [1]:
from gssutils import *

scraper = Scraper('https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/datasets/uktradeingoodsbyindustrycountryandcommodityexports')
scraper

## UK trade in goods by industry, country and commodity, exports

This new experimental dataset providing a breakdown of UK trade in goods by industry, country and commodity on a balance of payments basis. Data are subject to disclosure control, which means some data have been suppressed to protect confidentiality of individual traders.

### Distributions

1. UK trade in goods by industry, country and commodity, exports ([MS Excel Spreadsheet](https://www.ons.gov.uk/file?uri=/economy/nationalaccounts/balanceofpayments/datasets/uktradeingoodsbyindustrycountryandcommodityexports/2008to2016/tigindpublicationexportsv2.xlsx))


In [2]:
tabs = {tab.name: tab for tab in scraper.distribution().as_databaker()}
tabs.keys()

dict_keys(['CONTENTS', 'tigind_publication_exports'])

In [3]:
tabs = scraper.distribution().as_databaker()

In [4]:
tab = next(t for t in tabs if t.name =='tigind_publication_exports')

In [5]:
country = tab.filter(contains_string('country')).fill(DOWN).is_not_blank().is_not_whitespace()

In [6]:
industry = tab.filter(contains_string('industry')).fill(DOWN).is_not_blank().is_not_whitespace()

In [7]:
commodity = tab.filter(contains_string('commodity')).fill(DOWN).is_not_blank().is_not_whitespace()

In [8]:
year = tab.excel_ref('A1').fill(RIGHT).is_not_blank().is_not_whitespace().is_number()

In [9]:
observations = year.fill(DOWN).is_not_blank().is_not_whitespace()

In [10]:
Dimensions = [
            HDim(year,'Period',DIRECTLY,ABOVE),
            HDim(country,'ONS Partner Geography',DIRECTLY,LEFT),
            HDim(commodity,'CORD SITC',DIRECTLY,LEFT),
            HDim(industry,'SIC Industry',DIRECTLY,LEFT),
            HDimConst('Measure Type', 'GBP Total'),
            HDimConst('Unit', 'gbp-million'),
            HDimConst('Flow', 'exports')
            ]
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
table = c1.topandas()

In [11]:
import numpy as np
table['OBS'].replace('', np.nan, inplace=True)
table.dropna(subset=['OBS'], inplace=True)
if 'DATAMARKER' in table.columns:
    table.drop(columns=['DATAMARKER'], inplace=True)
table.rename(columns={'OBS': 'Value'}, inplace=True)
table['Value'] = table['Value'].astype(int)
table['Value'] = table['Value'].map(lambda x:'' if x == '...' else x )

In [12]:
for col in table.columns:
    if col not in ['Value', 'Period']:
        table[col] = table[col].astype('category')
        display(col)
        display(table[col].cat.categories)

'ONS Partner Geography'

Index(['AD Andorra', 'AE United Arab Emirates', 'AF Afghanistan',
       'AG Antigua & Barbuda', 'AI Anguilla', 'AL Albania', 'AM Armenia',
       'AO Angola', 'AQ Antarctica', 'AR Argentina',
       ...
       'VN Vietnam', 'VU Vanuatu', 'W1  Worldwide', 'WF Wallis & Futuna',
       'WS Samoa', 'XK Kosovo', 'YE Yemen', 'ZA South Africa', 'ZM Zambia',
       'ZW Zimbabwe'],
      dtype='object', length=235)

'CORD SITC'

Index(['0 Food & live animals', '1 Beverages & tobacco', '2 Crude materials',
       '3 Fuels', '4 Animal & vegetable oils & fats', '5 Chemicals',
       '6 Material manufactures', '7 Machinery & transport equipment',
       '8 Miscellaneous manufactures', '9 Unspecified goods', 'T Total'],
      dtype='object')

'SIC Industry'

...

Index(['GBP Total'], dtype='object')

'Unit'

Index(['gbp-million'], dtype='object')

'Flow'

Index(['exports'], dtype='object')

In [13]:
table['CORD SITC'].cat.categories = table['CORD SITC'].cat.categories.map(lambda x: x[:1])
table['ONS Partner Geography'].cat.categories = table['ONS Partner Geography'].cat.categories.map(lambda x: x[:2])
table['SIC Industry'].cat.categories = table['SIC Industry'].cat.categories.map(lambda x: x[:2])
display(table['CORD SITC'].cat.categories)
display(table['ONS Partner Geography'].cat.categories)
display(table['SIC Industry'].cat.categories)

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'T'], dtype='object')

Index(['AD', 'AE', 'AF', 'AG', 'AI', 'AL', 'AM', 'AO', 'AQ', 'AR',
       ...
       'VN', 'VU', 'W1', 'WF', 'WS', 'XK', 'YE', 'ZA', 'ZM', 'ZW'],
      dtype='object', length=235)

Index(['01', '02', '03', '05', '06', '07', '08', '09', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '30', '31', '32', '33', '35', '36', '37', '38',
       '39', '41', '42', '43', '45', '46', '47', '49', '50', '51', '52', '53',
       '55', '56', '58', '59', '60', '61', '62', '63', '65', '66', '68', '69',
       '70', '71', '72', '73', '74', '75', '77', '78', '79', '80', '81', '82',
       '84', '85', '86', '87', '88', '90', '91', '92', '93', '94', '95', '96',
       '98', '99'],
      dtype='object')

In [14]:
table['Period'] = 'year/' + table['Period'].astype(str).str[0:4]

In [15]:
table = table[['ONS Partner Geography', 'Period','Flow','CORD SITC', 'SIC Industry', 'Measure Type','Value','Unit' ]]

In [16]:
table

,ONS Partner Geography,Period,Flow,CORD SITC,SIC Industry,Measure Type,Value,Unit
0,W1,year/2008,exports,0,01,GBP Total,326,gbp-million
1,W1,year/2009,exports,0,01,GBP Total,319,gbp-million
2,W1,year/2010,exports,0,01,GBP Total,315,gbp-million
3,W1,year/2011,exports,0,01,GBP Total,391,gbp-million
4,W1,year/2012,exports,0,01,GBP Total,312,gbp-million
5,W1,year/2013,exports,0,01,GBP Total,354,gbp-million
6,W1,year/2014,exports,0,01,GBP Total,344,gbp-million
7,W1,year/2015,exports,0,01,GBP Total,424,gbp-million
8,W1,year/2016,exports,0,01,GBP Total,316,gbp-million
10,W1,year/2009,exports,1,01,GBP Total,1,gbp-million
